In [1]:
!pip install /home/ma-user/work/LSTM-GAF/mindnlp-0.1.1-py3-none-any.whl

Looking in indexes: http://pip.modelarts.private.com:8888/repository/pypi/simple
Processing /home/ma-user/work/step_into_chatgpt/mindnlp-0.1.1-py3-none-any.whl
mindnlp is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
DEPRECATION: moxing-framework 2.1.7.dc1f3d0b has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of moxing-framework or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [2]:
!pip install regex

Looking in indexes: http://pip.modelarts.private.com:8888/repository/pypi/simple
DEPRECATION: moxing-framework 2.1.7.dc1f3d0b has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of moxing-framework or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [1]:
import math

from mindspore import nn
from mindspore import ops
from mindspore.common.initializer import initializer
from mindspore import nn
from mindspore.ops import operations as P
from mindnlp import load_dataset, process, Vocab
from mindnlp.abc import Seq2vecModel
from mindnlp.engine import Trainer
from mindnlp.metrics import Accuracy
from mindnlp.modules import Glove, RNNEncoder, StaticLSTM,Fasttext
from mindnlp.transforms import BasicTokenizer
embed=Fasttext.from_pretrained('1M', 300, special_tokens=["<unk>", "<pad>"])
embed_size=300
lstm_hidden_size=200
num_layers=2
dense_hidden_size=200
num_classes=8
bidirectional=True
batch_size=32


In [ ]:
from model import SentimentNet

In [3]:
model=SentimentNet(10000,
                 embed_size,
                 200,
                 2,
                 bidirectional,
                 num_classes,
                 embed,
                 batch_size)


In [4]:
from mindspore.ops import functional as F, composite as C, operations as P
_atan_opt = C.MultitypeFuncGraph("atan_opt")
map=C.Map()
@_atan_opt.register("Tensor")
def atan_p(x):
    a=P.Atan()
    return 0.2*a(x*10)
class sgd_act(nn.SGD):
    
    def construct(self,g):
        
        g=map(_atan_opt,g)
        return super().construct(g)

In [5]:

import mindspore as ms
import mindspore
from mindspore.dataset import text, GeneratorDataset, transforms
from mindspore import nn, context
from mindnlp.transforms import PadTransform
from mindnlp.transforms.tokenizers import BertTokenizer
from mindnlp.engine import Trainer, Evaluator
from mindnlp.engine.callbacks import CheckpointCallback, BestModelCallback
from mindnlp.metrics import Accuracy
import os
from mindspore import ops, ms_function
from mindspore.amp import init_status, all_finite, DynamicLossScaler
import matplotlib
import matplotlib.pyplot as plt
import time
import numpy as np
from mindspore.train.serialization import save_checkpoint
#from tqdm import tqdm
from mindnlp.models import BertForSequenceClassification
from mindnlp._legacy.amp import auto_mixed_precision
from mindnlp.modules import Accumulator
#import GAF
from mindnlp.models import BertModel
import mindnlp.transforms.tokenizers as t
import pandas as pd
import json
import moxing as mox
from mindspore.communication.management import init, get_local_rank_size
data_dir='/home/ma-user/work/LSTM-GAF'
model_dir='/home/ma-user/work/LSTM-GAF/model'
class R8Dataset:
    def __init__(self, path):
        self.path = path
        self._labels, self._text_a = [], []
        self._load()
    def _load(self):
        self.df = pd.read_csv(self.path)
        labels ={'ship': 0,
                  'money-fx': 1,
                  'grain': 2,
                  'acq': 3,
                  'trade': 4,
                  'earn': 5,
                  'crude': 6,
                  'interest': 7}
        self.df['intent'] = [labels[i] for i in self.df['intent']]

    def __getitem__(self, index):
        return self.df['text'][index],self.df['intent'][index]

    def __len__(self):
        return len(self.df)

def process_dataset(source, tokenizer,pad_value, batch_size=32,shuffle=True):
    column_names = ["text_a","label"]
    rename_columns = ["input_ids","label"]

    dataset = GeneratorDataset(source, column_names=column_names, shuffle=shuffle)
    # transforms
    pad_op = PadTransform(64, pad_value=pad_value)
    type_cast_op = transforms.TypeCast(mindspore.int32)

    # map dataset
    dataset = dataset.map(operations=[tokenizer, pad_op], input_columns="text_a")
    dataset = dataset.map(operations=[type_cast_op], input_columns="label")
    # rename dataset
    dataset = dataset.rename(input_columns=column_names, output_columns=rename_columns)
    # batch dataset
    dataset = dataset.batch(batch_size)

    return dataset

tokenizer = BertTokenizer.from_pretrained(data_dir)
pad_value = tokenizer.token_to_id('[PAD]')
dataset_train = process_dataset(R8Dataset(data_dir + "/r8/r8-train-stemmed.csv"),
                                tokenizer, pad_value)
dataset_val = process_dataset(R8Dataset(data_dir + "/r8/r8-dev-stemmed.csv"), tokenizer,
                              pad_value)
print(dataset_val.get_dataset_size())
dataset_test = process_dataset(R8Dataset(data_dir + "/r8/r8-test-stemmed.csv"), tokenizer,
                               pad_value, shuffle=False)
gaf = False
# set bert config and define parameters for training
reslut_record = {'loss': [], 'acc': []}
total = dataset_train.get_dataset_size()
loss_list = []
acc_list=[]
#model = BertForSequenceClassification.from_pretrained(model_dir)
#model = auto_mixed_precision(model, 'O1')
loss_fn = nn.CrossEntropyLoss()
# n_model=BertForSequenceClassificationWithLoss(model,loss_fn)

optimizer = sgd_act(model.trainable_params(),
              learning_rate=1e-1,weight_decay=1e-8)  #



metric = Accuracy()
'''# define callbacks to save checkpoints
ckpoint_cb = CheckpointCallback(save_path='checkpoint', ckpt_name='bert_emotect', epochs=1, keep_checkpoint_max=2)
best_model_cb = BestModelCallback(save_path='checkpoint', ckpt_name='bert_emotect_best', auto_load=True)
trainer = Trainer(network=model, train_dataset=dataset_train,
                  eval_dataset=dataset_val, metrics=metric,
                  epochs=5, loss_fn=loss_fn, optimizer=optimizer, callbacks=[ckpoint_cb, best_model_cb],
                  jit=False)'''

loss_scaler = DynamicLossScaler(scale_value=2 ** 10, scale_factor=2, scale_window=1000)


INFO:root:Using MoXing-v2.1.7.dc1f3d0b-dc1f3d0b
INFO:root:Using OBS-Python-SDK-3.20.9.1


18


In [6]:
from inspect import signature
from tqdm.autonotebook import tqdm
from mindspore import log, mutable
from mindnlp import ms_jit
from mindnlp.abc import Metric
from mindnlp.engine.callbacks.callback_manager import CallbackManager, RunContext      
import numpy as np
from mindnlp.metrics.utils import _check_onehot_data, _check_shape, _convert_data_type
from mindnlp.abc import Metric
def eval_(model,dataset):
    model.set_train(False)
    _correct_num=0
    _total_num=0
    #if(True):
    with tqdm(total=dataset.get_dataset_size()) as progress:
        progress.set_description('Evaluate')
        for batch_idx, (input_ids, labels) in enumerate(dataset.create_tuple_iterator()):
            #print(input_ids)
            model.set_train(False)
            # = self._data_process(data, tgt_columns)
            outputs = model(input_ids)
            #print(outputs[0].shape)
            preds = outputs[0]
            y_pred = _convert_data_type(preds)#np
            y_true = _convert_data_type(labels)
            if y_pred.shape[1] !=8:
                raise ValueError(f'For `Accuracy.update`, class numbers do not match. Last input '
                             f'predicted data contain {self._class_num} classes, but current '
                             f'predicted data contain {y_pred.shape[1]} classes. Please check '
                             f'your predicted value (`preds`).')

            if   y_pred.ndim == y_true.ndim and \
                (_check_onehot_data(y_true) or y_true[0].shape == (1,)):
                y_true = y_true.argmax(axis=1)
        
            _check_shape(y_pred, y_true, 8)

            
            indices = y_pred.argmax(axis=1)

            res = (np.equal(indices, y_true) * 1).reshape(-1)

            _correct_num += res.sum()
            _total_num += res.shape[0]
            #print(indices.shape,y_true.shape)
            #print('end')
            #break
            progress.update(1)
    progress.close()
    print(f'Evaluate Score: {_correct_num/_total_num}')
    return _correct_num/_total_num,_correct_num,_total_num,

  0%|          | 0/18 [00:00<?, ?it/s]

Evaluate Score: 0.30895795246800734


(0.30895795246800734, 169, 547)

In [ ]:
def forward_fn(inputs, labels):
    logits = model(inputs, labels=labels)
    loss = logits[0]
    return loss#loss_scaler.scale(loss)
grad_fn = ops.value_and_grad(forward_fn, None, optimizer.parameters)
@ms_function
def train_step(data, label):
    status = init_status()
    data = ops.depend(data, status)
    loss, grads = grad_fn(data, label)
    #loss = loss_scaler.unscale(loss)
    #is_finite = all_finite(grads, status)
    #if is_finite:
    #    grads = loss_scaler.unscale(grads)
    #    loss = ops.depend(loss, optimizer(grads))
    #loss = ops.depend(loss, loss_scaler.adjust(is_finite))
    loss=ops.depend(loss, optimizer(grads))
    return loss#, is_finite
for epoch in range(100):
        print(f'=========epoch:{epoch}============')
        #eval_()
        loss_total = 0
        loss_100 = 0
        cur_step_nums = 0
        for batch_idx, (input_ids, labels) in enumerate(dataset_train.create_tuple_iterator()):
            cur_step_nums += 1
            loss= train_step(input_ids, labels)
            loss_total += loss
            loss_100 += float(loss)
            print(loss)
            if cur_step_nums % 25 == 0:
                #print(loss_100/100)
                loss_list.append(loss_100 / 25)
                loss_100 = 0
        if epoch % 9 == 0:
            save_checkpoint(model, model_dir+f'/mindspore_{epoch}.ckpt')
            
            #evaluator = Eval(network=model, eval_dataset=dataset_test, metrics=metric)
            #evaluator.run(tgt_columns="label")
            re=eval_(model,dataset_val)
            print(f"Accuracy:{re}")
            acc_list.append(re[0])
reslut_record['loss'].append(loss_list)
reslut_record['acc'].append(acc_list)
with open(save_dir+f'/result_bi_{device_id}.json','w+') as f:
    json.dump(reslut_record,fp=f,ensure_ascii=False,indent=True)
    f.close()

=========epoch:0============


[WARNING] ME(42718:281472905284160,MainProcess):2023-09-24-09:02:46.216.390 [mindspore/ops/primitive.py:713] The "_check_is_tensor" is a constexpr function. The input arguments must be all constant value.


2.0712876
2.0091085
1.9798882
1.9092603
1.8746021
1.8275031
1.816402
1.714988
1.6890595
1.5874345
1.5591667
1.6473758
1.6345496
1.5402845
1.5618216
1.406357
1.3493248
1.3261576
1.219505
1.597412
1.2788111
1.3816656
1.460716
1.249065
1.1703901
1.3970408
1.1472116
1.3929379
1.3095739
1.3571892
1.4135907
1.2776966
1.2081547
1.7127953
1.1001886
1.298137
1.4463729
1.253911
1.1576469
1.3677717
1.2407587
1.5054302
1.1567504
1.01128
1.3976117
1.096992
1.2937355
1.2215755
1.0818287
1.1657112
1.0419075
1.0759702
0.96168184
1.4129457
1.1058178
1.2847927
1.5058352
1.0626255
1.6488557
1.502825
0.897563
1.1202954
0.9625006
1.0248871
1.5622585
1.1974678
1.5849758
1.1718442
1.175292
1.1107249
1.2231371
1.2072322
1.1575584
1.2368383
1.299114
1.0199075
1.1539974
1.0209402
1.0465444
1.3460472
1.1045272
1.3231732
1.581898
1.1670456
1.399936
1.1395221
1.3551433
1.1478202
1.2404432
0.95108956
1.0037178
1.408308
1.1608243
1.3387747
1.3947004
1.3964437
1.4577777
1.3468386
1.336415
1.2615951
1.2804165
1.320693

[WARNING] ME(42718:281472905284160,MainProcess):2023-09-24-09:07:50.939.834 [mindspore/ops/primitive.py:713] The "_check_is_tensor" is a constexpr function. The input arguments must be all constant value.


0.8212861


...

Evaluate Score: 0.7385740402193784
Accuracy:(0.7385740402193784, 404, 547)
=========epoch:28============
0.30984038
0.6510681
0.4208587
0.33491433
0.5523913
0.61646676
0.54042256
0.52678883
0.90985644
0.84230655
0.4600472
0.83717364
0.60796106
0.41270438
0.48260272
0.46558803
0.45641327
0.5639425
0.7620083
0.58916783
0.4564045
0.34781235
0.34274697
0.9139298
0.35746545
0.70007145
0.33641312
0.54411805
0.41680852
0.4640299
0.63678265
0.3184953
0.41048276
0.25667763
0.67092395
0.1752738
0.41748905
0.64833045
0.41840625
0.58951366
0.51640975
0.31313705
0.32075956
0.37975365
0.5107431
0.5670814
0.45785388
0.31445044
0.4281041
0.16443828
0.16843438
0.11425991
0.36287013
0.6837789
0.8849191
0.45508337
0.86448324
0.4834281
0.44953477
0.057606988
0.42352772
0.45841703
0.41009593
0.23774257
0.5339285
0.30878472
0.5465128
0.36882484
0.205855
0.3033389
0.3052966
0.4487842
0.48421067
0.20243362
0.5081451
0.4352643
0.44447684
0.44901007
0.44452754
0.39355046
0.38639742
0.24358895
0.58150333
0.39500

  0%|          | 0/18 [00:00<?, ?it/s]

Evaluate Score: 0.7970749542961609
Accuracy:(0.7970749542961609, 436, 547)
=========epoch:37============
0.13160858
0.47460985
0.50313514
0.3663293
0.3685281
0.19931775
0.3547213
0.18662454
0.16251153
0.037970565
0.3771301
0.17657219
0.10594362
0.31631273
0.20462409
0.18526329
0.5505402
0.3288337
0.2675789
0.21522824
0.54880494
0.38112727
0.16728993
0.29497623
0.3235733
0.1851746
0.22713676
0.32006317
0.340016
0.30380362
0.48075807
0.4371466
0.20517635
0.35143468
0.31059015
0.06848177
0.393282
0.36405388
0.16321284
0.23442918
0.47734174
0.2490739
0.25300646
0.24280152
0.30576783
0.31472433
0.33741605
0.3214135
0.11384932
0.41033235
0.17997284
0.20395516
0.16612403
0.554119
0.27883652
0.1916879
0.31647655
0.22411929
0.48442996
0.1150952
0.47188595
0.42278665
0.25486493
0.34297866
0.3351732
0.37035897
0.23072755
0.38985968
0.37313575
0.18281716
0.21402165
0.2513272
0.28994977
0.79166067
0.3059737
0.4137081
0.44642755
0.1421398
0.32575905
0.38263583
0.8302652
0.3763149
0.25177184
0.500089

  0%|          | 0/18 [00:00<?, ?it/s]

Evaluate Score: 0.8062157221206582
Accuracy:(0.8062157221206582, 441, 547)
=========epoch:46============
0.09171596
0.06825307
0.2992146
0.25966486
0.2456946
0.042492032
0.15181816
0.2812596
0.025223248
0.14364755
0.22211584
0.26340118
0.4688187
0.13446781
0.19070072
0.09052217
1.4018409
0.32300895
0.07574746
0.2911597
0.2275293
0.1572418
0.038036108
0.19650024
0.21849404
0.25979793
0.17096286
0.13121682
0.11631657
0.08613834
0.14352521
0.086626425
0.03024266
0.263817
0.21073103
0.32767302
0.1260865
0.3835895
0.03593582
0.11065309
0.18654415
0.11346662
0.3387611
0.6886238
0.12382966
0.09274877
0.099131584
0.04748255
0.2204221
0.096910745
0.30970955
0.30299032
0.40665483
0.15121497
0.12477329
0.04831592
0.23704273
0.10345095
0.12321857
0.22082543
0.20144871
0.12489874
0.06524824
0.12277472
0.17540833
0.09598474
0.23233089
0.07624901
0.10589015
0.21532834
0.19159926
0.26528823
0.08434669
0.26841256
0.074190184
0.31107748
0.20186321
0.24356437
0.09258648
0.049148396
0.14729363
0.06007816


In [ ]:
def t(*x):
    print(*x)
t(1)

In [ ]:
dataset_train = process_dataset(R8Dataset(data_dir + "/r8/r8-train-stemmed.csv"),
                                tokenizer, pad_value)

for batch_idx, (input_ids, labels) in enumerate(dataset_train.create_tuple_iterator()):
    #model(input_ids)
    #break
    print(model(input_ids,labels)[1])
    break

In [ ]:
w=nn.LSTM(embed_size, lstm_hidden_size,
                            num_layers=num_layers, batch_first=True,
                            bidirectional=True)

In [ ]:
[1,2,3][1:3:1]

In [ ]:
import mindspore as ms
import numpy as np
net = ms.nn.LSTM(300, 16, 2, has_bias=True, batch_first=True, bidirectional=True)
x = ms.Tensor(np.ones([30, 5, 300]).astype(np.float32))
h0 = ms.Tensor(np.ones([1 * 2, 3, 16]).astype(np.float32))
c0 = ms.Tensor(np.ones([1 * 2, 3, 16]).astype(np.float32))
output, (hn, cn) = model(x)
print(output.shape)

In [ ]:
c= nn.LSTM(input_size=300,
                               hidden_size=200,
                               num_layers=3,
                               has_bias=True,
                               batch_first=True,
                               bidirectional=True,
                               dropout=0.0)
print(model.encoder(x))

In [ ]:
c

In [ ]:
model.encoder

In [ ]:
model.encoder